In [1]:
import cv2
from keras.models import load_model
import numpy as np
import time
import random
from enum import IntEnum


class Action(IntEnum):
    Rock = 0
    Paper = 1
    Scissors = 2


def get_computer_selection():
    selection = random.randint(0, 2)
    return selection


def get_player_selection(frame):
    player_action = 0

    resized_frame = cv2.resize(frame, (224, 224), interpolation = cv2.INTER_AREA)
    image_np = np.array(resized_frame)
    normalized_image = (image_np.astype(np.float32) / 127.0) - 1 # Normalize the image
    data[0] = normalized_image
    prediction = model.predict(data)

    if prediction[0][0] > .7:
        player_action = 0

    elif prediction[0][1] > .7:
        player_action = 1

    elif prediction[0][2] > .7: 
        player_action = 2

    return player_action


def countdown(t=3, frame):
    while t:
        if t == 3:
            print("ROCK")
            cv2.putText(frame, "ROCK", (10, 50),  cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 0), 1)

        if t == 2:
            print("PAPER")
            cv2.putText(frame, "PAPER", (10, 50),  cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 0), 1)
            
        if t == 1:
            print("SCISSORS")
            cv2.putText(frame, "SCISSORS", (10, 50),  cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 0), 1)

        time.sleep(1)
        t -= 1
    print('SHOOT')
    cv2.putText(frame, "SHOOT", (10, 50),  cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 0), 1)



def play_result(player_action, computer_action):
    victories = {
        Action.Rock: [Action.Scissors],  # Rock beats scissors
        Action.Paper: [Action.Rock],  # Paper beats rock
        Action.Scissors: [Action.Paper]  # Scissors beats paper
    }

    defeats = victories[player_action]
    if player_action == computer_action:
        print(f"Both players selected {player_action.name}. It's a tie!")
        return 0
    elif computer_action in defeats:
        print(f"{player_action.name} beats {computer_action.name}! You win!")
        return 1
    else:
        print(f"{computer_action.name} beats {player_action.name}! You lose.")
        return 2


def play_game():
    game_score = [0, 0]
        
    while True:
        ret, frame = cap.read()
        if not ret:
            continue
        
        #cv2.putText(frame, "TEST", (10, 50),  cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 0), 1)
        cv2.imshow('frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        countdown(3, frame)

        resized_frame = cv2.resize(frame, (224, 224), interpolation = cv2.INTER_AREA)
        image_np = np.array(resized_frame)
        normalized_image = (image_np.astype(np.float32) / 127.0) - 1 # Normalize the image
        data[0] = normalized_image
        prediction = model.predict(data)

        player_choice = 0

        if prediction[0][0] > .7:
            player_choice = 0

        elif prediction[0][1] > .7:
            player_choice = 1

        elif prediction[0][2] > .7: 
            player_choice = 2

        player_action = Action(player_choice)
        computer_action = Action(get_computer_selection())
        print(f"\nYou chose {player_action.name}, \
            computer chose {computer_action.name}.\n")

        result = play_result(player_action, computer_action)

        if result == 1:
            i = 0
        elif result == 2:
            i = 1
        else:
            continue

        game_score[i] += 1

        if game_score[0] == 3 or game_score[1] == 3:
            if game_score[0] == 3:
                print("YOU WIN THE MATCH", game_score)
            else:
                print("CPU WIN THE MATCH", game_score)
            break

        print(game_score)
    
    cap.release()
    # Destroy all the windows
    cv2.destroyAllWindows()    


model = load_model('keras_model.h5')
cap = cv2.VideoCapture(0)
ret = cap.set(cv2.CAP_PROP_FRAME_WIDTH, 600.0) 
ret = cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 300.0)
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)


play_game()


2022-02-01 21:07:36.733331: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ROCK
PAPER
SCISSORS
SHOOT

You chose Paper,             computer chose Scissors.

Scissors beats Paper! You lose.
[0, 1]
ROCK
PAPER
SCISSORS
SHOOT

You chose Scissors,             computer chose Scissors.

Both players selected Scissors. It's a tie!
ROCK
PAPER
SCISSORS
SHOOT

You chose Paper,             computer chose Scissors.

Scissors beats Paper! You lose.
[0, 2]
ROCK
PAPER
SCISSORS
SHOOT

You chose Scissors,             computer chose Scissors.

Both players selected Scissors. It's a tie!
ROCK
PAPER
SCISSORS
SHOOT

You chose Scissors,             computer chose Scissors.

Both players selected Scissors. It's a tie!
ROCK
PAPER
SCISSORS
SHOOT

You chose Scissors,             computer chose Paper.

Scissors beats Paper! You win!
[1, 2]
ROCK
PAPER
SCISSORS
SHOOT

You chose Scissors,             computer chose Paper.

Scissors beats Paper! You win!
[2, 2]
ROCK
PAPER
SCISSORS
SHOOT

You chose Scissors,             computer chose Scissors.

Both players selected Scissors. It's a tie!
RO